## A Demonstration of the Elliptic Curve (EC) Cryptosystem

The EC cryptosystem demonstrated below combines Diffie-Hellman style symmetric key agreement with a digital signature algorithm to achieve confidentiality, integrity and authentication of a message exchanged between two parties (Alice and Bob) over an insecure channel. Practically speaking, a third party (Eve) eavesdropping on the insecure channel has no way of learning the content of the message, nor any way of corrupting the message without the receiving party knowing it has been corrupted.

The key-agreement aspect of the cryptosystem provides message confidentiality, and is known by the initials ECDH, which short for Elliptic Curve Diffie-Hellman (alternatively ECDHE, the last *E* standing for *Ephemeral* to indicate that the session key negotiated by the message-exchanging parties is to be used only once and then discarded). The digital signature and verification aspect of the cryptosystem provides message integrity and authenticity, and is known by the initials ECDSA, for Elliptic Curve Digital Signature Algorithm.

Elliptic curve cryptography represents a novel approach to the discrete logarithm problem (DLP) first introduced by Diffie and Hellman in their 1976 paper [*New Directions in Cryptography*](https://ee.stanford.edu/~hellman/publications/24.pdf), which remains to this day the basis for modern day key negotiation over insecure channels. ECDLP has the advantage of requiring much shorter key lengths than traditional DH, however, to achieve equivalent security levels. This makes EC the only practical choice in applications on small, low-power devices, like contactless smart cards and smart phones. While an EC key of 256 bits in length provides 128-bit security (today considered the minimum acceptible security level), DH and RSA keys must be at least 3,072 bits to provide the same level of security.

### The Code
The following sequence of function calls to the `ec` module (located in the [/src](https://github.com/dchampion/crypto/tree/master/code/src) directory of this repository) illustrates a session in which Alice signs and encrypts a private message, and sends the message to Bob over an insecure channel. Meanwhile, Eve&mdash;a passive adversary&mdash;intercepts all the information exchanged between Alice and Bob on the insecure channel.

It is assumed that Alice, Bob and Eve all possess their own copy of the `ec` module; but that only Alice and Bob possess the signing and encryption keys produced by this module to achieve confidentiality, integrity and authentication of the private message Alice transmits to Bob.

Values below surrounded in square brackets [ ] are public (i.e., they can be transmitted over the insecure channel with no compromise to the security of the system, even if they are observed by Eve), and those that are not in square brackets are private (i.e., they must not be transmitted over the insecure channel).

In [63]:
import sys
sys.path.append("../code/src")
import ec
sys.path.append("../code/test")
import sym

Import the `ec` module, and a module `sym` that implements the symmetric cipher Alice and Bob will use in the session.

In [64]:
ec.new_curve(17,2,2,5,1,19,1,5)

The `ec` module defines a NIST-/SECG-sanctioned ellipctic curve by default; specifically, the secp256k1 curve (we know this curve is secure because it has secured the Bitcoin blockchain since 2009). For the purposes of this demonstration, however, Alice will define a much smaller (and therefore much less secure) curve.

The parameters required to define the new curve, in order, are the prime modulus (17), the coefficient a (2), the coefficient b (2), the x-coordinate of the base point (5), the y-coordinate of the base point (1), the number of points on the curve (19), the cofactor of the curve point group (1), and an optional parameter that specifies the number of iterations to test the curve's strength against a certain class of attacks (5) (the parameter is required here because the default value of 100 will cause the curve to fail validation).

If any of these parameters is invalid, `ec.new_curve` will raise an exception. As it turns out, a curve with these parameters is valid (albeit wholly insecure due to its small size).

In [65]:
print(ec._p, ec._a, ec._b, ec._Gx, ec._Gy, ec._n, ec._h)

17 2 2 5 1 19 1


In [85]:
pts = []
pt = ec._G
while pt != ec._i:
    pts.append(pt)
    pt = ec.add(ec._G, pt)
pts.append(pt)
print(pts)

[[5, 1], [6, 3], [10, 6], [3, 1], [9, 16], [16, 13], [0, 6], [13, 7], [7, 6], [7, 11], [13, 10], [0, 11], [16, 4], [9, 1], [3, 16], [10, 11], [6, 14], [5, 16], [None, None]]


To get a better picture of what's going on here, the loop in the above code snippet collects all the points in the curve and then prints them out. Note the first point in the list, the base point `[5, 1]`, whose coordinates Alice specified in the 4<sup>th</sup> and 5<sup>th</sup> parameters to `ec.new_curve`. Also note the last point, `[None, None]`, which on an elliptic curve is called the *point at infinity*, and denotes the identity element in an elliptic&ndash;curve point group.

The number of points in the group is 19, which Alice specified in the 6<sup>th</sup> parameter to `ec.new_curve`.

Finally, note that this code is not required for the protocol; it is only included to provide a visual representation of the points on this ellipctic curve.

In [68]:
dA, QA = ec.generate_keypair()

Next, Alice generates a key pair, and stores the keys in *dA* and *[QA]*; these are her private and public keys, respectively. The suffixed capital letter *A* denotes that these keys belong to Alice.

Alice transmits her public key *[QA]* to Bob, but she must keep her private key *dA* secret.

In [69]:
print(dA, QA)

14 [9, 1]


If we print the keys, we see the private key is an integer, and the public key a point on the curve. Note that the private key is the 1-based index of the curve in the group we printed above; that is, it is the *dA*<sup>th</sup> point in the list.

More formally, using elliptic curve math, we have *added* the base point `[5, 1]` to itself *dA* times to get the public key *QA*.

In [70]:
dB, QB = ec.generate_keypair()

Now it's Bob's turn to generate a key pair (suffixed with the capital letter *B* to denote they belong to him). As with Alice's keys, Bob transmits his public key *[QB]* to Alice, and keeps his private key *dB* secret.

In [71]:
print(dB, QB)

3 [10, 6]


If we print Bob's keys, we see a familiar pattern.

In [72]:
kSessionA = ec.generate_session_key(dA, QB)

Now Alice computes a session key `kSessionA`, using her private key *dA* and Bob's public key *QB*.

If you are familiar with traditional Diffie-Hellman, you should see something familiar here. The essential property of DH is expressed in the interface `ec.generate_session_key(dA, QB)`; namely, a secret session key is computed using one party's private key in combination with the other party's public key.

Whereas in traditional DH the private key is multiplied by the public key to produce an integer exponent to which to raise a common integer base, in ECDH the private key (an integer) is *multiplied* by the public key (a curve point). The result of both operations is an identical session key; in the former this is an integer, and in the latter it is a curve point (*multiplication* is another way of saying repeated addition, which is in fact the only operator supported by elliptic curve point groups).

An implementation detail is worth noting here. One might wonder how it is that a curve point `[x, y]`, generated by the multiplication of an integer (*dA*) to a base point (*QB*), is converted to a byte array that can be used as a key (*kSessionA*) in a symmetric cipher. The answer is that the x-coordinate&mdash;which is in fact just an integer in the range `0 < x < p`, where *p* is the prime modulus in the curve group&mdash;is extracted from the point, and just it is used for the session key.

In [73]:
kSessionB = ec.generate_session_key(dB, QA)

Now Bob computes a session key *kSessionB*, using his private key *dB* in combination with Alice's public key *QA*. If the essential property of DH holds, Alice's and Bob's session keys should be identical.

Further, Eve, having only observed Alice's and Bob's public keys, cannot feasibly derive the session key with just knowledge of the public keys. The only way to accomplish this would be to add curve points repeatedly, starting with the base point, until the session key is found. This would would take thousands of years at current computer speeds.

In [88]:
print(kSessionA == kSessionB)

True


We see here that *kSessionA* is in fact equivalent to *kSessionB*.

In [75]:
mA = "Sign and encrypt me!"

Now Alice composes a message for Bob, and stores it in *mA*.

In [76]:
S = ec.sign(dA, mA)

Alice signs the message and stores the signature in *[S]*.

In [78]:
mAC = sym.encrypt(kSessionA, mA)

Alice encrypts the message using her session key, and stores the ciphertext in *[mAC]*. Alice then transmits the ciphertext *[mAC]*, along with the signature *[S]*, to Bob.

In [80]:
mB = sym.decrypt(kSessionB, mAC)

Having received the ciphertext and signature from Alice, Bob first decrypts the ciphertext *[mAC]*, using his session key *kSessionB*, and stores the result in *mB*.

In [81]:
print(mB)

Sign and encrypt me!


Here we print *mB*, and see that it matches the plaintext *mA* Alice encrypted into *[mAC]* and transmitted to Bob. Note Alice transmitted *[mAC]* to Bob, not *mA*.

Thus far Alice has transmitted her message confidentially to Bob; what remains is for Bob to verify the integrity and authenticity of the message.

In [82]:
print(ec.verify(QA, mB, S))

True


Bob verifies the integrity (meaning the message signed by Alice was not altered) and authenticity (meaning Alice's private key *dA* was used to sign the message) of the message using the function `ec.verify`, passing it Alice's public key *[QA]*, the plaintext of the message *mB*, and the signature *S*.

If the function returns `True`, Bob can be satisfied that the conditions of confidentiality, integrity and authenticity have 